The coin change problem is commonly seen at Facebook and Amazon interviews (allegedly). You’re given coins of different denominations and a total amount of money needing to be represented with the coinage. From that, you need to write a function to compute the least number of coins that you need to make up that amount. If you can’t reach the given amount of money with any combination of the coins, you return -1. 

In [1]:
import numpy as np

#### Test coins

In [2]:
UScoins = [1,5,10,25]
Frenchcoins = [1,2,5,10,20,50]
Alien_coins = [3,7,26,73]
Mermaidcoins = [12,13,15,35,60]

#### Basic function for US coinage

In [3]:
def us_coin_change(coins:list,ttl:int):
    left = ttl
    selected_coins = []
    while left > 0:
        remainders = [left-c if left-c>=0 else left for c in coins]
        
        if min(remainders)==left:
            return -1
        
        chosen_coin_idx = remainders.index(min(remainders))
        selected_coins.append(coins[chosen_coin_idx])
        left -= coins[chosen_coin_idx]
        
        if sum(selected_coins)>ttl: # runs over total
            return -1
    
    return {'number of coins':len(selected_coins),'coins':selected_coins,'remainder':ttl-np.sum(selected_coins)}

In [4]:
print(us_coin_change(UScoins,153))
print(us_coin_change(UScoins,37))
print(us_coin_change(UScoins,84))
print()
print(us_coin_change(Frenchcoins,153))
print(us_coin_change(Frenchcoins,37))
print(us_coin_change(Frenchcoins,84))
print()
# this fails because it chooses 7 first, which makes it impossible to get to 12
# the less efficient, but correct path is required, 3+3+3
print(us_coin_change(Alien_coins,12))

{'number of coins': 9, 'coins': [25, 25, 25, 25, 25, 25, 1, 1, 1], 'remainder': 0}
{'number of coins': 4, 'coins': [25, 10, 1, 1], 'remainder': 0}
{'number of coins': 8, 'coins': [25, 25, 25, 5, 1, 1, 1, 1], 'remainder': 0}

{'number of coins': 5, 'coins': [50, 50, 50, 2, 1], 'remainder': 0}
{'number of coins': 4, 'coins': [20, 10, 5, 2], 'remainder': 0}
{'number of coins': 5, 'coins': [50, 20, 10, 2, 2], 'remainder': 0}

-1


#### Slightly less basic function

In [5]:
def change(coins:list,ttl:int):
    left = ttl
    selected_coins = []
    selected_factors = []
    partial_sum, use_single_coin = 0, False
    while left != 0:
        
        factors = [left//c if left//c>0 else 0 for c in coins]
        
        if max(factors)!=0:
            min_factor_idx = factors.index(min([f for f in factors if f > 0]))
            max_factor_idx = factors.index(max([f for f in factors if f > 0]))
        
            min_factor = factors[min_factor_idx]
            max_factor = factors[max_factor_idx]

            if max_factor*coins[max_factor_idx]==ttl:
                use_single_coin = True 
                single_factor = max_factor
                single_coin = coins[max_factor_idx]

            partial_sum += min_factor*coins[min_factor_idx]

            selected_coins.append(coins[min_factor_idx])
            selected_factors.append(min_factor)

            left -= min_factor*coins[min_factor_idx]

            if partial_sum==ttl:
                break
        else:
            if use_single_coin:
                return {'number of coins':1,'coin factor':single_factor,
                        'coins':single_coin,'remainder':ttl-(single_factor*single_coin)}
            else:
                return -1
            
    return {'number of coins':len(selected_coins),'coin factor':selected_factors,
            'coins':selected_coins,'remainder':ttl-partial_sum}

In [6]:
print(change(UScoins,12))
print(change(UScoins,204))
print(change(UScoins,63))
print(change(UScoins,43))
print()
print(change(Alien_coins,12))
print(change(Alien_coins,204))
print(change(Alien_coins,63))
print(change(Alien_coins,43))
print()
print(change(Frenchcoins,12))
print(change(Frenchcoins,204))
print(change(Frenchcoins,63))
print(change(Frenchcoins,43))

{'number of coins': 2, 'coin factor': [1, 2], 'coins': [10, 1], 'remainder': 0}
{'number of coins': 2, 'coin factor': [8, 4], 'coins': [25, 1], 'remainder': 0}
{'number of coins': 3, 'coin factor': [2, 1, 3], 'coins': [25, 10, 1], 'remainder': 0}
{'number of coins': 4, 'coin factor': [1, 1, 1, 3], 'coins': [25, 10, 5, 1], 'remainder': 0}

{'number of coins': 1, 'coin factor': 4, 'coins': 3, 'remainder': 0}
{'number of coins': 3, 'coin factor': [2, 2, 2], 'coins': [73, 26, 3], 'remainder': 0}
{'number of coins': 1, 'coin factor': 21, 'coins': 3, 'remainder': 0}
{'number of coins': 3, 'coin factor': [1, 2, 1], 'coins': [26, 7, 3], 'remainder': 0}

{'number of coins': 2, 'coin factor': [1, 1], 'coins': [10, 2], 'remainder': 0}
{'number of coins': 2, 'coin factor': [4, 2], 'coins': [50, 2], 'remainder': 0}
{'number of coins': 4, 'coin factor': [1, 1, 1, 1], 'coins': [50, 10, 2, 1], 'remainder': 0}
{'number of coins': 3, 'coin factor': [2, 1, 1], 'coins': [20, 2, 1], 'remainder': 0}


This made up coinage does poorly if the min factor is the same for different coins. I could adapt my current function to this scenario but I think I'll just implement a different/better/more flexible method that can take any kind of absurd, made-up, nonsense coinage and still work. 

In [7]:
print(change(Mermaidcoins,26))
print(change(Mermaidcoins,30))

-1
-1


#### Iterating vector method
I'm ignoring the instructions to return a -1 if the sum cannot be represented with the given denomination of coins. Instead I return the remainder.

In [8]:
def fcoin(coins,ttl):
    factors = np.array([[1,2,3,4,5,6,7,8,9,10] for _ in range(len(coins))]).T # factors
    left = np.array([ttl]*len(coins))
    m,n=factors.shape
    
    chfL = []
    chcL = []
    for i in range(n):
        coins.sort()
        coins = coins[::-1]
        fa = factors*coins
        dif = (left - fa)[:,i]
        idx = np.argsort(dif)
        pidx = dif[idx]>=0
        if any(pidx):
            chf = factors[idx][pidx][0,i]
            chc = coins[i]
            chfL.append(chf)
            chcL.append(chc)

            left = left - (chf*chc)

            left[left<0] = 0

        else:
            continue
            
    est = np.sum(np.array(chfL)*np.array(chcL))

    return {'number of coins':np.sum(chfL),'factors':chfL,'coins':chcL,'remainder':ttl-est}


In [9]:
UScoins = [1,5,10,25]
print(f'total given {267} for {fcoin(UScoins,267)}')
print(f'total given {7} for {fcoin(UScoins,7)}')
print(f'total given {73} for {fcoin(UScoins,73)}')
print(f'total given {49} for {fcoin(UScoins,49)}')

total given 267 for {'number of coins': 14, 'factors': [10, 1, 1, 2], 'coins': [25, 10, 5, 1], 'remainder': 0}
total given 7 for {'number of coins': 3, 'factors': [1, 2], 'coins': [5, 1], 'remainder': 0}
total given 73 for {'number of coins': 7, 'factors': [2, 2, 3], 'coins': [25, 10, 1], 'remainder': 0}
total given 49 for {'number of coins': 7, 'factors': [1, 2, 4], 'coins': [25, 10, 1], 'remainder': 0}


In [10]:
Alien_coins = [3,7,26,73]
print(f'total given {267} for {fcoin(Alien_coins,267)}')
print(f'total given {7} for {fcoin(Alien_coins,7)}')
print(f'total given {73} for {fcoin(Alien_coins,73)}')
print(f'total given {49} for {fcoin(Alien_coins,49)}')

total given 267 for {'number of coins': 7, 'factors': [3, 1, 3], 'coins': [73, 26, 7], 'remainder': 1}
total given 7 for {'number of coins': 1, 'factors': [1], 'coins': [7], 'remainder': 0}
total given 73 for {'number of coins': 1, 'factors': [1], 'coins': [73], 'remainder': 0}
total given 49 for {'number of coins': 4, 'factors': [1, 3], 'coins': [26, 7], 'remainder': 2}


In [11]:
Mermaidcoins = [12,13,15,35,60]
print(f'total given {267} for {fcoin(Mermaidcoins,267)}')
print(f'total given {7} for {fcoin(Mermaidcoins,7)}')
print(f'total given {73} for {fcoin(Mermaidcoins,73)}')
print(f'total given {49} for {fcoin(Mermaidcoins,49)}')

total given 267 for {'number of coins': 6, 'factors': [4, 1, 1], 'coins': [60, 15, 12], 'remainder': 0}
total given 7 for {'number of coins': 0.0, 'factors': [], 'coins': [], 'remainder': 7.0}
total given 73 for {'number of coins': 2, 'factors': [1, 1], 'coins': [60, 13], 'remainder': 0}
total given 49 for {'number of coins': 2, 'factors': [1, 1], 'coins': [35, 13], 'remainder': 1}


In [12]:
Frenchcoins = [1,2,5,10,20,50]
print(f'total given {267} for {fcoin(Frenchcoins,267)}')
print(f'total given {7} for {fcoin(Frenchcoins,7)}')
print(f'total given {73} for {fcoin(Frenchcoins,73)}')
print(f'total given {49} for {fcoin(Frenchcoins,49)}')

total given 267 for {'number of coins': 8, 'factors': [5, 1, 1, 1], 'coins': [50, 10, 5, 2], 'remainder': 0}
total given 7 for {'number of coins': 2, 'factors': [1, 1], 'coins': [5, 2], 'remainder': 0}
total given 73 for {'number of coins': 4, 'factors': [1, 1, 1, 1], 'coins': [50, 20, 2, 1], 'remainder': 0}
total given 49 for {'number of coins': 5, 'factors': [2, 1, 2], 'coins': [20, 5, 2], 'remainder': 0}
